In [10]:
import sys
import math
from gurobipy import *
import numpy
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
alpha = 0.0001
pi = 0.5
T = 20
N = 10000

def impact(x,alpha,pi):
    return 1-alpha*x**pi

def deri_impact(x,alpha,pi):
    return -alpha*pi*x**(pi-1)

def gradcomp(n, gradient,x,alpha,pi):
    #part1 
    impact_series = np.cumprod(impact(x,alpha,pi))
    part1 = impact_series

    #part2
    #the rest multiplied terms
    intermediate_term1 = (impact_series*x)[::-1]
    term1 = np.cumsum(intermediate_term1)[::-1]/impact(x,alpha,pi)
    #derivative of impact functions
    term2 = deri_impact(x,alpha,pi)
    part2 = np.nan_to_num(term1) * np.nan_to_num(term2)
    gradient = part1 + part2
    return gradient

def myfunction(x):
    return (np.dot(impact(x,alpha,pi).cumprod(),x))

def line_search(x ,y):
    index = np.linspace(0,1,10000)
    change = (index[:,np.newaxis] * y)
    x_matrix = np.tile(x, (10000,1))
    sum_change = pd.DataFrame(x_matrix + change)
    best_size = sum_change.apply(myfunction, axis=1).argmax()
    return 1/10000 * best_size

def loop(n,xsol,row1):
    x = np.zeros(n)
    gradient = numpy.zeros(n)
    slacks = 0
    for j in range(n):
        x[j] = xsol[j]

    maxits = 100
    for itcount in range(maxits):
        val = myfunction(x)
        gradient = gradcomp(n, gradient,x,alpha,pi)
        print ("hello, value = ", val)
        compute_slacks(n, row1, x, slacks)
        y = steplp(n, row1, x, slacks, gradient)
        step_size = line_search(x,y)
        print('step_size:', step_size)
        
        val_1 = myfunction(x)                        #stopping condition
        x = x + step_size * y
        val_2 = myfunction(x)
        if val_2 - val_1 < 0.000000001:
            print('optimal achieved at ', val)
            print('optimal selling: ', x)
            return            
        print ('x:  ', x)

def compute_slacks(n, row1, x, slacks):
    slacks = N - np.sum(x)

    
def steplp(n, row1, x, slacks, gradient):
    m = Model("feasible")
    m.ModelSense = 1 #minimize
    #create variables and put into a dictionary
    deltavar = {}
    for j in range(n):
        deltavar[j] = m.addVar(lb = -x[j], ub = N-x[j], obj = -gradient[j], vtype = GRB.CONTINUOUS, name = "delta_"+str(j+1))

    # Update model to integrate new variables
    m.update()    
    #add first constraint
    xpr = LinExpr()
    for j in range(n):
        xpr += row1[j]*deltavar[j]
    m.addConstr(xpr == slacks, name="const1")        
   
    m.write("step.lp")
    m.optimize()

    code = "optimal"

    if m.status == GRB.status.INF_OR_UNBD:
        print('->LP infeasible or unbounded\n')
        code = "inf_or_unbd"
    if m.status == GRB.status.UNBOUNDED:
        print('->LP unbounded\n')
        code = "unbd"
    if m.status == GRB.status.INFEASIBLE:
        print('->LP infeasible\n')
        code = "infeas"


    print(" -> LP optimal, value = %g\n" %(m.objval))

    deltasol = numpy.zeros(n)
    for j in range(n):
        #print ("%s = %g" %(deltavar[j].varname, deltavar[j].x))
        deltasol[j] = deltavar[j].x

    print ('deltasol:', deltasol)

    return deltasol

def run():
    n = T
    row1 = numpy.repeat(1,n)
    feasible = np.repeat(N/T, T)
    #feasible = np.zeros(T)
    #feasible[0] = N
    code, xsol = 'feasible', feasible

    print ("initial feasible solution = ")
    print(xsol)

    loop(n,xsol,row1)

if __name__ == '__main__':
    run()




initial feasible solution = 
[500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500. 500. 500. 500. 500.]
hello, value =  9768.504658048427
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+02, 1e+04]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 20 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0433299e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective -1.043329874e+02
 -> LP optimal, value = -104.333

deltasol: [9500. -500. -500. -500. -500. -500. -500. -500. -500. -500. -500. -500.
 -500. -500. -500. -500. -500. -500. -500. -500.]
step_size: 0.9766
x:   [9777.7   11.7   11.7   11.7   11.7   11.7   11.7   11.7   11.7   11.7
   11.7   11.7   11.7   11.7   11.7   11.7   11.7   1

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e+00, 1e+04]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 20 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.7954365e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective -1.795436506e+00
 -> LP optimal, value = -1.79544

deltasol: [-8.75352754e+03  8.87621740e+03 -6.81610329e+00 -6.81610329e+00
 -6.81610329e+00 -6.81610329e+00 -6.81610329e+00 -6.81610329e+00
 -6.81610329e+00 -6.81610329e+00 -6.81610329e+00 -6.81610329e+00
 -6.81610329e+00 -6.81610329e+00 -6.81610329e+00 -6.81610329e+00
 -6.81610329e+00 -6.81610329e+00 -6.81610329e+00 -6.81610329e+00]
step_size: 0.013000000000000001
x:   [8.63973168e+03 1.23917343e+03 6.72749395e+00 6.72749395e+00
 6.72749395e+00 6.72749395e+00 6.72749395e+00 6.72749395e+00
 6.72749

step_size: 0.008
x:   [8.65407597e+03 1.24928510e+03 5.36882952e+00 5.36882952e+00
 5.36882952e+00 5.36882952e+00 5.36882952e+00 5.36882952e+00
 5.36882952e+00 5.36882952e+00 5.36882952e+00 5.36882952e+00
 5.36882952e+00 5.36882952e+00 5.36882952e+00 5.36882952e+00
 5.36882952e+00 5.36882952e+00 5.36882952e+00 5.36882952e+00]
hello, value =  9902.05004542283
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 1e+04]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 20 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5305935e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -1.530593479e-01
 -> LP optimal, value = -0.153059

deltasol: [ 1345.92403427 -1249.28510292    -5.36882952    -5.36882952
    -5.368829

       0   -1.1025575e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective -1.102557517e-01
 -> LP optimal, value = -0.110256

deltasol: [ 1338.41532548 -1255.36744874    -4.61377093    -4.61377093
    -4.61377093    -4.61377093    -4.61377093    -4.61377093
    -4.61377093    -4.61377093    -4.61377093    -4.61377093
    -4.61377093    -4.61377093    -4.61377093    -4.61377093
    -4.61377093    -4.61377093    -4.61377093    -4.61377093]
step_size: 0.0351
x:   [8.70856305e+03 1.21130405e+03 4.45182757e+00 4.45182757e+00
 4.45182757e+00 4.45182757e+00 4.45182757e+00 4.45182757e+00
 4.45182757e+00 4.45182757e+00 4.45182757e+00 4.45182757e+00
 4.45182757e+00 4.45182757e+00 4.45182757e+00 4.45182757e+00
 4.45182757e+00 4.45182757e+00 4.45182757e+00 4.45182757e+00]
hello, value =  9902.06919246182
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+00, 1e+04]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 20 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.4318483e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -5.431848296e-01
 -> LP optimal, value = -0.543185

deltasol: [-8.70288453e+03  8.77505271e+03 -4.00934345e+00 -4.00934345e+00
 -4.00934345e+00 -4.00934345e+00 -4.00934345e+00 -4.00934345e+00
 -4.00934345e+00 -4.00934345e+00 -4.00934345e+00 -4.00934345e+00
 -4.00934345e+00 -4.00934345e+00 -4.00934345e+00 -4.00934345e+00
 -4.00934345e+00 -4.00934345e+00 -4.00934345e+00 -4.00934345e+00]
step_size: 0.004
x:   [8.66807299e+03 1.26004750e+03 3.99330608e+00 3.99330608e+00
 3.99330608e+00 3.99330608e+00 3.99330608e+00 3.99330608e+00
 3.99330608e+00 3.9933

step_size: 0.0033
x:   [8.67063113e+03 1.26319329e+03 3.67642095e+00 3.67642095e+00
 3.67642095e+00 3.67642095e+00 3.67642095e+00 3.67642095e+00
 3.67642095e+00 3.67642095e+00 3.67642095e+00 3.67642095e+00
 3.67642095e+00 3.67642095e+00 3.67642095e+00 3.67642095e+00
 3.67642095e+00 3.67642095e+00 3.67642095e+00 3.67642095e+00]
hello, value =  9902.084139604198
Optimize a model with 1 rows, 20 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+00, 1e+04]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 20 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -6.4412997e-02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds
Optimal objective -6.441299741e-02
 -> LP optimal, value = -0.064413

deltasol: [ 1329.3688705  -1263.19329343    -3.67642095    -3.67642095
    -3.6764